This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Load and save data

In this tutorial, we will load and save electron backscatter diffraction (EBSD) patterns, Kikuchi master patterns generated with *EMsoft* and virtual backscatter electron (VBSE) images from and to various formats supported by kikuchipy.

## Load patterns

### From a file

kikuchipy can read and write experimental EBSD patterns and Kikuchi master patterns from or to many formats (see [supported formats](#Supported-file-formats)).
To load patterns from a file use, the [load()](../reference/generated/kikuchipy.load.rst) function.
Let's import the necessary libraries and read the nickel EBSD example dataset directly from file (not via [kikuchipy.data.nickel_ebsd_small()](../reference/generated/kikuchipy.data.nickel_ebsd_small.rst))

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

import os
from pathlib import Path
import tempfile

import dask.array as da
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt

import hyperspy.api as hs
import kikuchipy as kp

In [ ]:
data_path = Path("../../src/kikuchipy/data")
nordif_ebsd = "nordif/Pattern.dat"
s = kp.load(data_path / nordif_ebsd)
s

Or, load the stereographic projection of the upper hemisphere of an EBSD master pattern for a 20 keV beam energy from a modified version of *EMsoft*'s master pattern file, returned from their `EMEBSDmaster.f90` program

In [ ]:
emsoft_master_pattern = (
    "emsoft_ebsd_master_pattern/ni_mc_mp_20kv_uint8_gzip_opts9.h5"
)
s_mp = kp.load(data_path / emsoft_master_pattern)
s_mp

Both the stereographic and the square Lambert projections of this master pattern can be loaded with [kikuchipy.data.nickel_ebsd_master_pattern_small()](../reference/generated/kikuchipy.data.nickel_ebsd_master_pattern_small.rst).

All file readers support accessing the data's metadata without loading it into memory (with the [Dask library](https://docs.dask.org/en/latest)), which can be useful when processing large data sets, one data chunk at a time, to avoid memory errors

In [ ]:
s_lazy = kp.load(data_path / nordif_ebsd, lazy=True)
print(s_lazy)
s_lazy.data

Parts or all of the data can be read into memory by calling [compute()](http://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy._signals.lazy.html#hyperspy._signals.lazy.LazySignal.compute)

In [ ]:
s_lazy_copy = s_lazy.inav[:2, :].deepcopy()
s_lazy_copy.compute()
s_lazy_copy

In [ ]:
s_lazy.compute()
s_lazy

<div class="alert alert-info">

Note

When lazily loaded EBSD patterns are processed, they are processed chunk by chunk, which can lead to longer processing times, so processing lazy data sets should be done with some care.
See the relevant [HyperSpy user guide](http://hyperspy.org/hyperspy-doc/v1.7/user_guide/big_data.html) for some tips.

</div>

Patterns can be visualized by navigating the navigation space

In [ ]:
s.plot()

Upon loading, kikuchipy reads some scan information from the file and stores it in the `original_metadata` attribute

In [ ]:
# s.original_metadata  # Long output

Other information is stored in a standard location in the `metadata` attribute in fields specified in the [HyperSpy metadata structure](https://hyperspy.org/hyperspy-doc/v1.7/user_guide/metadata_structure.html); these are not used by methods in kikuchipy

In [ ]:
s.metadata

The number of patterns in horizontal and vertical direction, pattern size in pixels, scan step size and detector pixel size are stored in the `axes_manager` attribute

In [ ]:
print(s.axes_manager)

This information can be modified directly, and information in `metadata` and `axes_manager` can be modified by the [EBSD](../reference/generated/kikuchipy.signals.EBSD.rst) class methods [set_scan_calibration()](../reference/generated/kikuchipy.signals.EBSD.set_scan_calibration.rst) and [set_detector_calibration()](../reference/generated/kikuchipy.signals.EBSD.set_detector_calibration.rst).

In addition to the `metadata`, `original_metadata` and `axes_manager` properties, kikuchipy tries to read the following from the file if available:

- a [CrystalMap](https://orix.readthedocs.io/en/stable/reference/generated/orix.crystal_map.CrystalMap.html) with indexing results into the [xmap](../reference/generated/kikuchipy.signals.EBSD.xmap.rst) attribute

- an [EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst) describing the projection/pattern center (PC) values into the [detector](../reference/generated/kikuchipy.signals.EBSD.detector.rst) attribute

- a static background into the [static_background](../reference/generated/kikuchipy.signals.EBSD.static_background.rst) attribute.

The `xmap` attribute is `None` if it is not read, while some readers return an "empty" crystal map with points containing the identity rotation and the same phase ID of an undefined phase with no point group set.

In [ ]:
s.xmap

In [ ]:
s.xmap.rotations

In [ ]:
s.xmap.phase_id

In [ ]:
s.detector

In [ ]:
s.detector.pc

In [ ]:
s.static_background

<div class="alert alert-info">

Note

*New in version 0.8.*

The `EBSD` class inherits all methods from HyperSpy's [Signal2D](https://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy._signals.signal2d.html#hyperspy._signals.signal2d.Signal2D) class.
An effort is made to handle the mentioned attributes of `xmap`, `detector` and `static_background` when the navigation and/or signal dimensions are cropped, like when calling `EBSD.inav[]` or `EBSD.isig[]`.
This handling is new, and so needs time and more tests to mature.
It is considered experimental functionality.

</div>

### From a NumPy array

An `EBSD`, [EBSDMasterPattern](../reference/generated/kikuchipy.signals.EBSDMasterPattern.rst) or [ECPMasterPattern](../reference/generated/kikuchipy.signals.ECPMasterPattern.rst) (electron channeling pattern) signal can be created directly from a NumPy array.
To create a data set of (60 x 60) pixel patterns in a (10 x 20) grid, i.e. 10 and 20 patterns in the horizontal and vertical scan directions respectively, of random intensities

In [ ]:
s_np = kp.signals.EBSD(np.random.random((20, 10, 60, 60)))
s_np

### From a Dask array

When processing large data sets, it is useful to load data lazily with the Dask library.
This can be done upon reading patterns [from a file](#From-a-file) by setting `lazy=True` when using the `load()` function, or directly from a `dask.array.Array`

In [ ]:
s_da = kp.signals.LazyEBSD(
    da.random.random((20, 10, 60, 60), chunks=(2, 10, 60, 60))
)
print(s_da)
s_da.data

### From a HyperSpy signal

HyperSpy provides the method [set_signal_type()](https://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.set_signal_type) to change between [BaseSignal](https://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy.signal.html#hyperspy.signal.BaseSignal) subclasses, of which `EBSD`, `EBSDMasterPattern`, `ECPMasterPattern` and [VirtualBSEImage](../reference/generated/kikuchipy.signals.VirtualBSEImage.rst) are four.
To get one of these signals from a [HyperSpy Signal2D](https://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy._signals.signal2d.html#hyperspy._signals.signal2d.Signal2D) signal

In [ ]:
s_hs = hs.signals.Signal2D(np.random.random((20, 10, 60, 60)))
s_hs

In [ ]:
for signal_type in [
    "EBSD",
    "VirtualBSEImage",
    "ECPMasterPattern",
    "EBSDMasterPattern",
]:
    s_hs.set_signal_type(signal_type)
    print(s_hs)

## Save patterns

To save signals to file, use the [save()](../reference/generated/kikuchipy.signals.EBSD.save.rst) method.
For example, to save an `EBSD` signal in an HDF5 file in our default [h5ebsd format](#kikuchipy-h5ebsd), with file name `"patterns.h5"`

In [ ]:
temp_dir = Path(tempfile.mkdtemp())
s.save(temp_dir / "patterns")


<div class="alert alert-warning">

Warning

If we want to overwrite an existing file:

```python
s.save("patterns.h5", overwrite=True)
```

</div>

If we want to save patterns in [NORDIF's binary format](#NORDIF-binary) instead

In [ ]:
s.save(temp_dir / "patterns.dat")

To save an `EBSDMasterPattern` to an HDF5 file, we use the [save method inherited from HyperSpy](https://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy.signal.html#hyperspy.signal.BaseSignal.save) to write to their [HDF5 specification](https://hyperspy.org/hyperspy-doc/v1.7/user_guide/io.html#hspy-hyperspy-s-hdf5-specification) or [zarr specification](https://hyperspy.org/hyperspy-doc/v1.7/user_guide/io.html#zspy-hyperspy-s-zarr-specification)

In [ ]:
s_hs.save(temp_dir / "master_pattern.hspy")
s_hs

These master patterns can then be read into an `EBSDMasterPattern` signal again via HyperSpy's [load()](http://hyperspy.org/hyperspy-doc/v1.7/api/hyperspy.io.html#hyperspy.io.load)

In [ ]:
s_mp2 = hs.load(
    temp_dir / "master_pattern.hspy", signal_type="EBSDMasterPattern"
)
s_mp2

<div class="alert alert-info">

Note

To save results from statistical decomposition (machine learning) of patterns to file see the section [Saving and loading results](https://hyperspy.org/hyperspy-doc/v1.7/user_guide/mva.html#saving-and-loading-results) in HyperSpy's user guide.
Note that the file extension `.hspy` or `.zspy` must be used upon saving, `s.save('patterns.hspy')`, as the default extension in kikuchipy, `.h5`, yields a kikuchipy h5ebsd file where the decomposition results aren't saved.
The saved patterns can then be reloaded using HyperSpy's `load()` function passing the `signal_type="EBSD"` parameter [as explained above](#From-a-HyperSpy-signal).

</div>

## Supported file formats

Currently, kikuchipy has readers and writers for the following formats:

| Format                                                                    | Read | Write |
| ------------------------------------------------------------------------- | ---- | ----- |
| [EMsoft simulated EBSD HDF5](#EMsoft-simulated-EBSD-HDF5)                 | Yes  | No    |
| [EMsoft EBSD master pattern HDF5](#EMsoft-EBSD-master-pattern-HDF5)       | Yes  | No    |
| [EMsoft ECP master pattern HDF5](#EMsoft-ECP-master-pattern-HDF5)         | Yes  | No    |
| [EMsoft TKD master pattern HDF5](#EMsoft-TKD-master-pattern-HDF5)         | Yes  | No    |
| [kikuchipy h5ebsd](#kikuchipy-h5ebsd)                                     | Yes  | Yes   |
| [Bruker h5ebsd](#Bruker-h5ebsd)                                           | Yes  | No    |
| [EDAX h5ebsd](#EDAX-h5ebsd)                                               | Yes  | No    |
| [EDAX binary](#EDAX-binary)                                               | Yes  | No    |
| [NORDIF binary](#NORDIF-binary)                                           | Yes  | Yes   |
| [NORDIF calibration patterns](#NORDIF-calibration-patterns)               | Yes  | No    |
| [Oxford Instruments binary](#Oxford-Instruments-binary)                   | Yes  | No    |
| [Oxford Instruments h5ebsd (H5OINA)](#Oxford-Instruments-h5ebsd-(H5OINA)) | Yes  | No    |
| [Directory of EBSD patterns](#Directory-of-EBSD-patterns)                 | Yes  | No    |

<div class="alert alert-info">

Note

If you want to process your patterns with kikuchipy, but use an unsupported EBSD vendor software, or if you want to write your processed patterns to a vendor format that does not support writing, please request this feature in our [issue tracker](https://github.com/pyxem/kikuchipy/issues).

</div>

### EMsoft simulated EBSD HDF5

Dynamically simulated EBSD patterns returned by EMsoft's `EMEBSD.f90` program as HDF5 files can be read as an `EBSD` signal

In [ ]:
emsoft_ebsd = "emsoft_ebsd/simulated_ebsd.h5"  # Dummy data set
s_sim = kp.load(data_path / emsoft_ebsd)
s_sim

Here, the EMsoft simulated EBSD [file_reader()](../reference/generated/kikuchipy.io.plugins.emsoft_ebsd.file_reader.rst) is called, which takes the optional argument `scan_size`.
Passing `scan_size=(2, 5)` will reshape the pattern data shape from `(10, 10, 10)` to `(2, 5, 10, 10)`

In [ ]:
s_sim2 = kp.load(data_path / emsoft_ebsd, scan_size=(2, 5))
print(s_sim2)
print(s_sim2.data.shape)

Simulated EBSD patterns can be written to the [kikuchipy h5ebsd format](#kikuchipy-h5ebsd), the [NORDIF binary format](#NORDIF-binary), or to HDF5/zarr files using HyperSpy's HDF5/zarr specifications [as explained above](#Save-patterns).

### EMsoft EBSD master pattern HDF5

Master patterns returned by EMsoft's `EMEBSDmaster.f90` program as HDF5 files can be read as an [EBSDMasterPattern](../reference/generated/kikuchipy.signals.EBSDMasterPattern.rst) signal

In [ ]:
s_mp = kp.load(data_path / emsoft_master_pattern)

print(s_mp)
print(s_mp.projection)
print(s_mp.hemisphere)
print(s_mp.phase)

Here, the EMsoft EBSD master pattern [file_reader()](../reference/generated/kikuchipy.io.plugins.emsoft_ebsd_master_pattern.file_reader.rst) is called, which takes the optional arguments `projection`, `hemisphere` and `energy`.
The stereographic projection is read by default.
Passing `projection="lambert"` will read the square Lambert projection instead.
The upper hemisphere is read by default.
Passing `hemisphere="lower"` or `hemisphere="both"` will read the lower hemisphere projection or both, respectively.
Master patterns for all beam energies are read by default.
Passing `energy=(10, 20)` or `energy=15` will read the master pattern(s) with beam energies from 10 to 20 keV, or just 15 keV, respectively

In [ ]:
s_mp = kp.load(
    data_path / emsoft_master_pattern,
    projection="lambert",
    hemisphere="both",
    energy=20,
)

print(s_mp)
print(s_mp.projection)
print(s_mp.hemisphere)

Master patterns can be written to HDF5/zarr files using HyperSpy's HDF5/zarr specification [as explained above](#Save-patterns).

See <cite data-cite="jackson2019dictionary">Jackson et al. (2019)</cite> for a hands-on tutorial explaining how to simulate these patterns with EMsoft, and <cite data-cite="callahan2013dynamical">Callahan & De Graef (2013)</cite> for details of the underlying theory.

### EMsoft ECP master pattern HDF5

Master patterns returned by EMsoft's `EMECPmaster.f90` program as HDF5 files can be read as an [ECPMasterPattern](../reference/generated/kikuchipy.signals.ECPMasterPattern.rst) signal just like an EBSD master pattern.
The EMsoft ECP master pattern [file_reader()](../reference/generated/kikuchipy.io.plugins.emsoft_ecp_master_pattern.file_reader.rst) is used, which supports the same parameters as the [EMsoft EBSD master pattern reader](#EMsoft-EBSD-master-pattern-HDF5).

### EMsoft TKD master pattern HDF5

Master patterns returned by EMsoft's `EMTKDmaster.f90` program as HDF5 files can be read as an [EBSDMasterPattern](../reference/generated/kikuchipy.signals.EBSDMasterPattern.rst) signal just like an EBSD master pattern.
The EMsoft TKD master pattern [file_reader()](../reference/generated/kikuchipy.io.plugins.emsoft_tkd_master_pattern.file_reader.rst) is used, which supports the same parameters as the [EMsoft EBSD master pattern reader](#EMsoft-EBSD-master-pattern-HDF5).

### kikuchipy h5ebsd

The h5ebsd format <cite data-cite="jackson2014h5ebsd">Jackson et al. (2014)</cite> is based on the [HDF5 open standard](https://www.hdfgroup.org/solutions/hdf5/) (Hierarchical Data Format version 5).
HDF5 files can be read and edited using e.g. the HDF Group's reader [HDFView](https://www.hdfgroup.org/downloads/hdfview) or the Python package [h5py](https://docs.h5py.org/en/stable).
Upon loading an HDF5 file with extension .h5, .hdf5, or .h5ebsd, the correct reader is determined from the file.
Other supported h5ebsd formats are listed in the [table above](#Supported-file-formats).

If an h5ebsd file contains multiple scans, as many scans as desirable can be read from the file.
For example, if the file contains two scans with names `"Scan 1"` and `"Scan 2"`

In [ ]:
kikuchipy_ebsd = "kikuchipy_h5ebsd/patterns.h5"
s1, s2 = kp.load(
    data_path / kikuchipy_ebsd, scan_group_names=["Scan 1", "Scan 2"]
)
print(s1)
print(s2)

Here, the h5ebsd [file_reader()](../reference/generated/kikuchipy.io.plugins.kikuchipy_h5ebsd.file_reader.rst) is called.
If only `"Scan 1"` is to be read, `scan_group_names="Scan 1"` can be passed

In [ ]:
s1 = kp.load(data_path / kikuchipy_ebsd, scan_group_names="Scan 1")
s1

The `scan_group_names` parameter is unnecessary if only the first scan in the file is to be read, since reading only the first scan in the file is the default behaviour.

So far, only [saving patterns](#Save-patterns) to kikuchipy's own h5ebsd format is supported.
It is possible to write a new scan with a scan name `"Scan x"`, where `x` is an integer, to an existing, but closed, h5ebsd file in the kikuchipy format, e.g. one containing only `Scan 1`, by passing

In [ ]:
new_file = "patterns_new.h5"
s1.save(temp_dir / new_file, scan_number=1)
s2.save(temp_dir / new_file, add_scan=True, scan_number=2)

s1_new, s2_new = kp.load(
    temp_dir / new_file, scan_group_names=["Scan 1", "Scan 2"]
)
print(s1_new)
print(s2_new)

Here, the h5ebsd [file_writer()](../reference/generated/kikuchipy.io.plugins.kikuchipy_h5ebsd.file_writer.rst) is called.
The EBSD class attributes `xmap` (crystal map), `detector` (EBSD detector) and `static_background` are written to and read from file if available.

### Bruker h5ebsd

Bruker Nano's h5ebsd files with extension .h5, .hdf5, or .h5ebsd can be read.
Available parameters when calling `kikuchipy.load("bruker_patterns.h5")` are described in the [file_reader()](../reference/generated/kikuchipy.io.plugins.bruker_h5ebsd.file_reader.rst).
As with the kikuchipy h5ebsd reader, multiple scans can be read at once.

The projection center (PC) arrays are read into the `detector` attribute, and the static background pattern is read into the `static_background` attribute.
The orientation and phase data are so far not read, but note that this can be read using [orix](https://orix.readthedocs.io/en/stable/reference/generated/orix.io.plugins.bruker_h5ebsd.html).

### EDAX h5ebsd

EDAX's h5ebsd files with extension .h5, .hdf5, or .h5ebsd can be read.
Available parameters when calling `kikuchipy.load("edax_patterns.h5")` are described in the [file_reader()](../reference/generated/kikuchipy.io.plugins.edax_h5ebsd.file_reader.rst).
As with the kikuchipy h5ebsd reader, multiple scans can be read at once.

The projection center (PC) arrays are read into the `detector` attribute.
The orientation and phase data are so far not read.

### EDAX binary

Patterns stored in the EDAX TSL's binary UP1/2 file format, with intensities as 8-bit or 16-bit unsigned integer, can be read. File version 1 and >= 3 are supported.

In [ ]:
edax_binary_path = data_path / "edax_binary/"  # Directory with dummy signals
s_edax = kp.load(edax_binary_path / "edax_binary.up1")
s_edax

Here, the EDAX binary [file_reader()](../reference/generated/kikuchipy.io.plugins.edax_binary.file_reader.rst) is called.

Files with version 1 has no information on the navigation (map) shape, so we can pass this to the reader in the `nav_shape` parameter

In [ ]:
s_edax2 = kp.load(edax_binary_path / "edax_binary.up1", nav_shape=(3, 3))
s_edax2

Patterns acquired in an hexagonal grid *can* be read, but the returned signal will have only one navigation dimension

In [ ]:
s_edax3 = kp.load(edax_binary_path / "edax_binary.up2")
s_edax3

### NORDIF binary

Patterns acquired using NORDIF's acquisition software are stored in a binary file usually named "Pattern.dat".
Scan information is stored in a separate text file usually named "Setting.txt", and both files usually reside in the same directory.
If this is the case, the patterns can be loaded by passing the file name as the only parameter.
If this is not the case, the setting file can be passed upon loading

In [ ]:
s_nordif = kp.load(
    data_path / nordif_ebsd, setting_file=data_path / "nordif/Setting.txt"
)
s_nordif

Here, the NORDIF [file_reader()](../reference/generated/kikuchipy.io.plugins.nordif.file_reader.rst) is called.
If the scan information, i.e. scan and pattern size, in the setting file is incorrect or the setting file is not available, patterns can be loaded by passing

In [ ]:
s_nordif = kp.load(
    data_path / nordif_ebsd, scan_size=(1, 9), pattern_size=(60, 60)
)
s_nordif

If a static background pattern named \"Background acquisition pattern.bmp\" is stored in the same directory as the pattern file, this is stored in `EBSD.static_background` upon loading

In [ ]:
s_nordif.static_background

Patterns can also be [saved to a NORDIF binary file](#Save-patterns), upon which the NORDIF [file_writer()](../reference/generated/kikuchipy.io.plugins.nordif.file_writer.rst) is called.
Note, however, that so far no new setting file, background pattern, or calibration patterns are created upon saving.

### NORDIF calibration patterns

NORDIF calibration patterns in bitmap format named "Calibration (x,y).bmp", where "x" and "y" correspond to coordinates listed in the NORDIF setting file, usually named "Setting.txt", can be loaded

In [ ]:
s_nordif_cal = kp.load(data_path / "nordif/Setting.txt")
s_nordif_cal

Here, the NORDIF calibration patterns [file_reader()](../reference/generated/kikuchipy.io.plugins.nordif_calibration_patterns.file_reader.rst)  is called.
Lazy loading is not supported for this reader, thus the `lazy` parameter is not used.

If a static background pattern named "Background calibration pattern.bmp" is stored in the same directory as the pattern file, this is stored in `EBSD.static_background` upon loading.

The pixel indices of the calibration patterns into the full electron image (named "Area.bmp" by the NORDIF software) are read into the `original_metadata`.
The shapes of this image and the region of interest (ROI) of the scanned area within this image are also attempted to be read.
All this information can be useful when obtaining a plane of projection centers (PCs) for all patterns in the full dataset based on these calibration patterns (see other user guide tutorials for this).

### Oxford Instruments binary

Uncompressed patterns stored in the Oxford Instruments binary .ebsp file format, with intensities as 8-bit or 16-bit unsigned integer, can be read

In [ ]:
oxford_binary_path = data_path / "oxford_binary"
s_oxford = kp.load(oxford_binary_path / "patterns.ebsp")
s_oxford

Here, the Oxford Instruments binary [file_reader()](../reference/generated/kikuchipy.io.plugins.oxford_binary.file_reader.rst) is called.

Every pattern's flattened index into the 2D navigation map, as well as their entry in the file (map order isn't always the same as file order) can be retrieved from `s_oxford.original_metadata.map1d_id` and `s_oxford.original_metadata.file_order`, respectively.
The following data may be read as well, depending on their presence in the file:

* `s_oxford.original_metadata.beam_x`: Every pattern's column in microns
* `s_oxford.original_metadata.beam_y`: Every pattern's row in microns
* `s_oxford.original_metadata.map_x`: Every pattern's column
* `s_oxford.original_metadata.map_y`: Every pattern's row

All these are 1D arrays.

In [ ]:
s_oxford.original_metadata

If the beam positions aren't present in the file, the returned signal will have a single navigation dimension the size of the number of patterns.

Files with only the non-indexed patterns can also be read. The returned signal will then have a single navigation dimension the size of the number of patterns.
The flattened index into the 2D navigation map mentioned above can be useful to determine the location of each non-indexed pattern.

### Oxford Instruments h5ebsd (H5OINA)

Oxford Instruments' h5ebsd files (H5OINA) with extension .h5oina can be read.
Available parameters when calling `kikuchipy.load("patterns.h5oina")` are described in the [file_reader()](../reference/generated/kikuchipy.io.plugins.oxford_h5ebsd.file_reader.rst).
As with the kikuchipy h5ebsd reader, multiple scans can be read at once.

The projection center (PC) arrays are read into the `detector` attribute and the static background pattern is read into the `static_background` attribute.
The orientation and phase data are so far not read.

Oxford Instruments allow writing both processed and unprocessed patterns.
To read the unprocessed patterns, pass `processed=False` when loading.

### Directory of EBSD patterns

Many EBSD patterns in image files in a directory can be read as an `EBSD` signal, assuming all images with that file extension have the same shape and data type. Valid formats are those [supported by imageio](https://imageio.readthedocs.io/en/stable/formats/index.html).

To demonstrate how the images can be read, we will first write nine patterns to a temporary with their filenames formatted a certain way

In [ ]:
temp_dir2 = Path(tempfile.mkdtemp())
s = kp.data.nickel_ebsd_small()
y, x = np.indices(s.axes_manager.navigation_shape[::-1])
y = y.ravel()
x = x.ravel()
s.unfold_navigation_space()
for i in range(s.axes_manager.navigation_size):
    iio.imwrite(temp_dir2 / f"pattern_x{x[i]}y{y[i]}.tif", s.data[i])

# Print file contents
os.listdir(temp_dir2)

If filenames are formatted like this `_x0y0.tif` or `-0-0.png`, we do not have to specify this file name pattern when reading the images

In [ ]:
s1 = kp.load(temp_dir2 / "*.tif")
s1

Here [file_reader()](../reference/generated/kikuchipy.io.plugins.ebsd_directory.file_reader.rst) of this plugin was called.
If filenames are formatted some other way, we have to pass this as a [regular expression](https://docs.python.org/3/library/re.html) to `xy_pattern`

In [ ]:
s2 = kp.load(
    temp_dir2 / "*.tif",
    xy_pattern=r"_x(\d+)y(\d+).tif",
    show_progressbar=True,
)
s2

Here we also printed a progressbar when reading the patterns from file, which can be useful when we want to read a large number of images.

## From kikuchipy into other software

Patterns saved in the [h5ebsd format](#kikuchipy-h5ebsd) can be read by the dictionary indexing and related routines in [EMsoft](https://github.com/EMsoft-org/EMsoft) using the `EMEBSD` reader.
Those routines in EMsoft also have a `NORDIF` reader.

Patterns saved in the [h5ebsd format](#kikuchipy-h5ebsd) can of course be read in Python like any other HDF5 data set

In [ ]:
import h5py

with h5py.File(data_path / kikuchipy_ebsd, mode="r") as f:
    dset = f["Scan 2/EBSD/Data/patterns"]
    print(dset)
    patterns = dset[()]
    print(patterns.shape)
    plt.figure()
    plt.imshow(patterns[0], cmap="gray")
    plt.axis("off")

## Load and save virtual BSE images

One or more virtual backscatter electron (BSE) images in a [VirtualBSEImage](../reference/generated/kikuchipy.signals.VirtualBSEImage.rst) signal can be read and written to file using one of HyperSpy's many readers and writers.

If they are only to be used internally in HyperSpy, they can be written to and read back from HyperSpy's HDF5/zarr specification [as explained above for EBSD master patterns](#Save-patterns).
If we want to write the images to image files, HyperSpy also provides a series of image readers/writers, as explained in their [IO user guide](https://hyperspy.org/hyperspy-doc/v1.7/user_guide/io.html#images).

Writing as a stack of TIFF images

In [ ]:
# Get virtual image from image generator
vbse_imager = kp.imaging.VirtualBSEImager(s)
print(vbse_imager)

print(vbse_imager.grid_shape)
vbse = vbse_imager.get_images_from_grid()
print(vbse)

In [ ]:
vbse.rescale_intensity()
vbse.unfold_navigation_space()  # 1D navigation space required for TIFF
vbse

In [ ]:
vbse_fname = "vbse.tif"
vbse.save(temp_dir / vbse_fname)  # Easily read into e.g. ImageJ

Read the TIFF stack back into a `VirtualBSEImage` signal

In [ ]:
vbse2 = hs.load(temp_dir / vbse_fname, signal_type="VirtualBSEImage")
vbse2

We can also write them to e.g. `png` or `bmp` files with `Matplotlib`

In [ ]:
nav_size = vbse.axes_manager.navigation_size
for i in range(nav_size):
    plt.imsave(temp_dir / f"vbse{i}.png", vbse.inav[i].data)

In [ ]:
# Remove files written to disk in this user guide
for file in os.listdir(temp_dir):
    os.remove(temp_dir / file)
for file in os.listdir(temp_dir2):
    os.remove(temp_dir2 / file)
os.rmdir(temp_dir)
os.rmdir(temp_dir2)